In [8]:
import copy
import time
import threading
import pandas as pd
import numpy as np
import more_itertools
import csv
import random
import itertools
pd.options.mode.chained_assignment = None  # default = 'warn'

In [9]:
product_rules = pd.read_csv('Data/master_rules.csv')
# product_rules.drop(columns=['Unnamed: 0'], inplace=True)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Instant food products,soda,0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,Instant food products,whole milk,0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,UHT-milk,whole milk,0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,abrasive cleaner,rolls/buns,0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,artif. sweetener,other vegetables,0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [10]:
with open('Data/names.csv') as f:
    read = csv.reader(f)
    names = list(read)

In [11]:
print(names)

[['Instant food products', 'UHT-milk', 'abrasive cleaner', 'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder', 'bathroom cleaner', 'beef', 'berries', 'beverages', 'bottled beer', 'bottled water', 'brandy', 'brown bread', 'butter', 'butter milk', 'cake bar', 'candles', 'candy', 'canned beer', 'canned fish', 'canned fruit', 'canned vegetables', 'cat food', 'cereals', 'chewing gum', 'chicken', 'chocolate', 'chocolate marshmallow', 'citrus fruit', 'cleaner', 'cling film/bags', 'cocoa drinks', 'coffee', 'condensed milk', 'cooking chocolate', 'cookware', 'cream', 'cream cheese ', 'curd', 'curd cheese', 'decalcifier', 'dental care', 'dessert', 'detergent', 'dish cleaner', 'dishes', 'dog food', 'domestic eggs', 'female sanitary products', 'finished products', 'fish', 'flour', 'flower (seeds)', 'flower soil/fertilizer', 'frankfurter', 'frozen chicken', 'frozen dessert', 'frozen fish', 'frozen fruits', 'frozen meals', 'frozen potato products', 'frozen vegetables', 'fruit/vegetable jui

In [12]:
print(len(names[0]))

167


In [13]:
product_rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

In [14]:
test = product_rules['antecedents'].iloc[0]
print(test)
test1 = product_rules['antecedents'].iloc[167]
print(test1)

Instant food products
 curd, UHT-milk


In [15]:
def change_format(x):
    delim = ','
    return frozenset(x.split(delim))

In [16]:
test = change_format(test)
print(test)
print(type(test))
test1 = change_format(test1)
print(test1)
print(type(test1))

frozenset({'Instant food products'})
<class 'frozenset'>
frozenset({' curd', ' UHT-milk'})
<class 'frozenset'>


In [17]:
product_rules['antecedents'] = product_rules['antecedents'].apply(change_format)
product_rules['consequents'] = product_rules['consequents'].apply(change_format)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [18]:
product_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24689,"( root vegetables, other vegetables, sausag...","( whole milk, rolls/buns)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069
24690,"( root vegetables, other vegetables, sausag...","( rolls/buns, soda)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124
24691,"( root vegetables, rolls/buns, soda, saus...","( whole milk, other vegetables)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189
24692,"( root vegetables, rolls/buns, whole milk,...","( other vegetables, soda)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947


In [144]:
def clean_up(factored_answer):
    to_check = list(factored_answer)
    index = 0
    while index < len(to_check):
        if to_check[index].find(','):
            to_split = copy.copy(to_check[index].split(','))
            to_check.pop(index)
            for j in range(len(to_split)):
                to_check.append(copy.copy(to_split[j]))
            index = 0
        else:
            index += 1
            continue
    return set(to_check)

def recommend(z):  # input is list of lists
    print("Z contains: ", z)
    print()
    print("Length of Z is: ",len(z))
    print()
    global product_rules
    master_query = None
    while True:
        try:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[3])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[4])))]
        except IndexError:
            print("Query after not having 5 entries is: ",master_query)
            print("Type of query is: ", type(master_query))
            print()
            pass
        if master_query is not None:
            break
        try:
            print("Made it to second try statement!!\n")
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[3])))]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))]
        except IndexError:
            return "OOPS!"
        if master_query is None:
            return None
    master_query.sort_values(by=['support','confidence','lift'],inplace=True,ascending=(False,False,False), ignore_index=True)
    # maybe update possible graph here ???
    master_query['consequents'] = master_query['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    results = list(master_query['consequents'])
    print(len(results))
    print(results)
    if len(results) == 0:
        return None
    else:
        return clean_up(results)
    # if len(results) == 1:
    #     print("GOT CAUGHT IN THE IF STATEMENT!!")
    #     return results
    # print('MADE IT PAST THE IF STATEMENT!!!')
    # factored_answer = set()
    # for value in range(len(results)):
    #     factored_answer.add(copy.copy(results[value]))
    # print("Factored answer: ",factored_answer)
    # print(len(factored_answer))
    # factored_answer_after_cleanup = clean_up(factored_answer)
    # return factored_answer_after_cleanup
    # df[df['antecedents'] == {'rolls/buns', 'Instant food products'}]

In [145]:
def synchronized(func):
    func.__lock__ = threading.Lock()

    def synced_func(*args, **kws):
        with func.__lock__:
            return func(*args, **kws)
    return synced_func

In [146]:
@synchronized
def start_here(x):
    master_search = []
    for j in range(1,6):
        for element in itertools.combinations(x, j):
            master_search.append(list(element))
    print("Length of master search is: ", len(master_search))
    while True:
        try:
            response = recommend(master_search[-1])
        except IndexError:
            return 'SOMETHING WENT WRONG!!!'
        if response is None or response == "OOPS!":
            master_search.pop()
            print("Length of Master Search is: ", len(master_search))
            print("Master search contains: ", master_search)
        else:
            # add code to check if items already in inventory... TODO
            break
    # print("Master Search Query Contains: ", master_search)
    # print('Length of Master Search Query: ', len(master_search))
    # print('\n\n\n')
    print('\n\n\n')
    print('Cleaned up version is: ', response)
    print("Length is: ", len(response))
    return response

In [147]:
product_rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
5,(artif. sweetener),(whole milk),0.007442,0.458045,0.004619,0.620690,1.355085,0.001210,1.428791
6,(bathroom cleaner),(other vegetables),0.004362,0.376700,0.002566,0.588235,1.561548,0.000923,1.513729
7,(bathroom cleaner),(whole milk),0.004362,0.458045,0.003079,0.705882,1.541078,0.001081,1.842648
8,(beef),(whole milk),0.119579,0.458045,0.064152,0.536481,1.171241,0.009379,1.169218
9,(bottled beer),(whole milk),0.158840,0.458045,0.085450,0.537964,1.174480,0.012694,1.172973


In [155]:
product_rules[product_rules['antecedents'] == {'chocolate marshmallow'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
27,(chocolate marshmallow),(whole milk),0.015396,0.458045,0.007698,0.5,1.091597,0.000646,1.083911


In [162]:
product_rules[(product_rules['antecedents'] == {'  rolls/buns'}) & (product_rules['antecedents'] == {'Instant food products'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [163]:
product_rules[product_rules['antecedents'] == {'  rolls/buns', 'Instant food products'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
132,"( rolls/buns, Instant food products)",(root vegetables),0.005389,0.230690,0.002823,0.523810,2.270618,0.001580,1.615550
133,"( rolls/buns, Instant food products)",(soda),0.005389,0.313318,0.003593,0.666667,2.127764,0.001904,2.060046


In [164]:
product_rules[product_rules['antecedents'] == {'Instant food products', '  rolls/buns'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
132,"( rolls/buns, Instant food products)",(root vegetables),0.005389,0.230690,0.002823,0.523810,2.270618,0.001580,1.615550
133,"( rolls/buns, Instant food products)",(soda),0.005389,0.313318,0.003593,0.666667,2.127764,0.001904,2.060046


In [160]:
product_rules.iloc[132]

antecedents           (  rolls/buns, Instant food products)
consequents                               (root vegetables)
antecedent support                                 0.005389
consequent support                                  0.23069
support                                            0.002823
confidence                                          0.52381
lift                                               2.270618
leverage                                            0.00158
conviction                                          1.61555
Name: 132, dtype: object

In [161]:
q = product_rules['antecedents'].iloc[132]
print(q)
print(type(q))
print()

frozenset({'  rolls/buns', 'Instant food products'})
<class 'frozenset'>


In [151]:
list_to_check_5 = []
for i in range(5):
    random_index = random.randrange(len(names[0]))
    list_to_check_5.append(copy.copy(names[0][random_index]))

In [152]:
print(list_to_check_5)

['brandy', 'frozen fruits', 'shopping bags', 'beef', 'grapes']


In [153]:
ans5 = start_here(list_to_check_5)

Length of master search is:  31
Z contains:  ['brandy', 'frozen fruits', 'shopping bags', 'beef', 'grapes']

Length of Z is:  5

0
[]
Length of Master Search is:  30
Master search contains:  [['brandy'], ['frozen fruits'], ['shopping bags'], ['beef'], ['grapes'], ['brandy', 'frozen fruits'], ['brandy', 'shopping bags'], ['brandy', 'beef'], ['brandy', 'grapes'], ['frozen fruits', 'shopping bags'], ['frozen fruits', 'beef'], ['frozen fruits', 'grapes'], ['shopping bags', 'beef'], ['shopping bags', 'grapes'], ['beef', 'grapes'], ['brandy', 'frozen fruits', 'shopping bags'], ['brandy', 'frozen fruits', 'beef'], ['brandy', 'frozen fruits', 'grapes'], ['brandy', 'shopping bags', 'beef'], ['brandy', 'shopping bags', 'grapes'], ['brandy', 'beef', 'grapes'], ['frozen fruits', 'shopping bags', 'beef'], ['frozen fruits', 'shopping bags', 'grapes'], ['frozen fruits', 'beef', 'grapes'], ['shopping bags', 'beef', 'grapes'], ['brandy', 'frozen fruits', 'shopping bags', 'beef'], ['brandy', 'frozen fru

KeyboardInterrupt: 

In [ ]:
print(ans5)

In [ ]:
print(len(ans5))

In [165]:
# print(type(ans5.pop()))

<class 'str'>


In [61]:
list_to_check_4 = []
for i in range(4):
    random_index = random.randrange(len(names[0]))
    list_to_check_4.append(copy.copy(names[0][random_index]))
print(list_to_check_4)

['butter milk', 'female sanitary products', 'long life bakery product', 'cling film/bags']


In [158]:
ans4 = start_here(list_to_check_4)

In [159]:
print(ans4)

{'pip fruit', 'whipped/sour cream', ' other vegetables, whole milk', ' rolls/buns, soda', 'pork', ' whole milk, soda', 'other vegetables', 'pastry', 'tropical fruit', 'other vegetables,  yogurt', 'root vegetables', 'whole milk', 'canned beer', 'soda', ' whole milk, bottled water', ' whipped/sour cream, whole milk', 'frankfurter', ' rolls/buns, whole milk', 'butter', 'beef', 'domestic eggs', 'citrus fruit', ' tropical fruit, whole milk', 'white bread', 'yogurt,  rolls/buns', 'bottled water', 'sausage', 'rolls/buns', ' yogurt, soda', 'other vegetables,  pip fruit', ' root vegetables, whole milk', 'curd', 'other vegetables,  rolls/buns', 'other vegetables,  whole milk', 'margarine', 'bottled beer', ' pork, whole milk', ' shopping bags, whole milk', 'yogurt', 'brown bread', 'newspapers', 'fruit/vegetable juice', ' other vegetables, bottled water', 'shopping bags', 'dessert', ' rolls/buns, shopping bags', ' other vegetables, soda', ' yogurt, whole milk'}


In [63]:
list_to_check_3 = []
for i in range(3):
    random_index = random.randrange(len(names[0]))
    list_to_check_3.append(copy.copy(names[0][random_index]))
print(list_to_check_3)

['cream', 'specialty bar', 'soap']


In [160]:
ans3 = start_here(list_to_check_3)

In [161]:
print(ans3)

{'pip fruit', 'whipped/sour cream', ' other vegetables, whole milk', ' rolls/buns, soda', 'pork', ' whole milk, soda', 'other vegetables', 'pastry', 'tropical fruit', 'other vegetables,  yogurt', 'root vegetables', 'whole milk', 'canned beer', 'soda', ' whole milk, bottled water', ' whipped/sour cream, whole milk', 'frankfurter', ' rolls/buns, whole milk', 'butter', 'beef', 'domestic eggs', 'citrus fruit', ' tropical fruit, whole milk', 'white bread', 'yogurt,  rolls/buns', 'bottled water', 'sausage', 'rolls/buns', ' yogurt, soda', 'other vegetables,  pip fruit', ' root vegetables, whole milk', 'curd', 'other vegetables,  rolls/buns', 'other vegetables,  whole milk', 'margarine', 'bottled beer', ' pork, whole milk', ' shopping bags, whole milk', 'yogurt', 'brown bread', 'newspapers', 'fruit/vegetable juice', ' other vegetables, bottled water', 'shopping bags', 'dessert', ' rolls/buns, shopping bags', ' other vegetables, soda', ' yogurt, whole milk'}


In [65]:
list_to_check_2 = []
for i in range(2):
    random_index = random.randrange(len(names[0]))
    list_to_check_2.append(copy.copy(names[0][random_index]))
print(list_to_check_2)

['cake bar', 'root vegetables']


In [162]:
ans2 = start_here(list_to_check_2)

KeyboardInterrupt: 

In [ ]:
print(ans2)

In [ ]:
list_to_check_1 = []
for i in range(1):
    random_index = random.randrange(len(names[0]))
    list_to_check_1.append(copy.copy(names[0][random_index]))
print(list_to_check_1)

In [ ]:
ans1 = start_here(list_to_check_1)

In [ ]:
print(ans1)

In [223]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [165]:
product_rules.to_csv('Data/master_rules.csv', index=False)

In [225]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
